In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, count, avg, sum, round, desc
from pyspark.sql.types import FloatType

df_itensProva_silver = spark.read.table("nttdataeducacao.silver.itensProva_tratados_2024")
df_itensProva_ouro = df_itensProva_silver

In [0]:
df_itensProva_ouro = df_itensProva_ouro.withColumnRenamed(
    "area_de_conhecimento_do_item", "area_extensa"
)

df_itensProva_ouro = (
    df_itensProva_ouro
        .withColumn("motivo_para_o_abandono_do_item", F.lower("motivo_para_o_abandono_do_item"))
        .withColumn("gabarito_do_item", F.upper("gabarito_do_item"))
)

In [0]:
display(df_itensProva_ouro.limit(15))

In [0]:
df_itensProva_ouro = df_itensProva_ouro.withColumn(
    "dificuldade_classificada",
    F.when(F.col("parametro_de_dificuldade") > 1.5, "Difícil")
     .when(F.col("parametro_de_dificuldade") > 0.5, "Médio")
     .when(F.col("parametro_de_dificuldade") > -1, "Fácil")
     .when(F.col("parametro_de_dificuldade") <= -1, "Muito fácil")
     .otherwise("Não classificado")
)

In [0]:
display(df_itensProva_ouro)

In [0]:
df_itensProva_ouro = df_itensProva_ouro.withColumn(
    "tipo_de_questao",
    F.when(F.col("gabarito_do_item") == "X", "Questão anulada")
     .when(F.col("indicador_de_item_abandonado") == 1, "Questão invalidada")
     .otherwise("Questão válida")
)

In [0]:
df_itensProva_ouro = df_itensProva_ouro.withColumn(
    "categoria_do_motivo",
    F.when(F.col("motivo_para_o_abandono_do_item") == "Algoritmo não convergiu", "Problema técnico")
     .when(F.col("motivo_para_o_abandono_do_item") == "Pedagógico", "Problema pedagógico")
     .when(F.col("motivo_para_o_abandono_do_item") == "Anulado no gabarito (X)", "Anulação oficial")
     .when(F.col("motivo_para_o_abandono_do_item") == "Não aplicável", "Não aplicável")
     .otherwise("Não especificado")
)


In [0]:
display(df_itensProva_ouro)

In [0]:
df_itensProva_ouro = df_itensProva_ouro.withColumn(
    "tipo_de_prova",
    F.when(F.col("item_pertencente_a_prova_adaptada") == 1, "Prova adaptada")
     .otherwise("Prova regular")
)

In [0]:
df_itensProva_ouro = df_itensProva_ouro.withColumn(
    "item_anulado",
    F.when(F.col("gabarito_do_item") == "X", 1).otherwise(0)
)

In [0]:
df_itensProva_ouro = df_itensProva_ouro.withColumn(
    "faixa_de_discriminacao",
    F.when(F.col("parametro_de_discriminacao") < 0.4, "Baixa")
     .when(F.col("parametro_de_discriminacao") < 0.8, "Média")
     .otherwise("Alta")
)

In [0]:
display(df_itensProva_ouro)


### **Resumo por área de conhecimento**

In [0]:
from pyspark.sql.functions import *

df_gold = df_itensProva_ouro.groupBy("area_extensa").agg(

    # Quantidade total de itens na área
    count("*").alias("total_itens"),
    
    # Itens anulados
    sum(col("item_anulado")).alias("total_anulados"),

    # Itens inválidos (indicador = 1)
    sum(col("indicador_de_item_abandonado")).alias("total_invalidos"),
    sum(when(col("motivo_para_o_abandono_do_item") == "Algoritmo não convergiu", 1).otherwise(0)).alias("motivo_algoritmo_nao_convergiu"),
    sum(when(col("motivo_para_o_abandono_do_item") == "Não aplicável", 1).otherwise(0)).alias("motivo_nao_aplicavel"),
    sum(when(col("motivo_para_o_abandono_do_item") == "Anulado no gabarito (x)", 1).otherwise(0)).alias("motivo_anulado_gabarito_x"),

    # Itens de prova adaptada
    sum(when(col("tipo_de_prova") == "Prova adaptada", 1).otherwise(0)).alias("total_itens_adaptados"),

    # Médias dos parâmetros TRI
    round(avg("parametro_de_dificuldade"), 3).alias("media_dificuldade"),
    round(avg("parametro_de_discriminacao"), 3).alias("media_discriminacao"),
    round(avg("parametro_de_acerto_ao_acaso"), 3).alias("media_acerto_ao_acaso"),
    
    # Contagem por classificação de dificuldade
    sum(when(col("dificuldade_classificada") == "Muito fácil", 1).otherwise(0)).alias("qtd_muito_facil"),
    sum(when(col("dificuldade_classificada") == "Fácil", 1).otherwise(0)).alias("qtd_facil"),
    sum(when(col("dificuldade_classificada") == "Médio", 1).otherwise(0)).alias("qtd_medio"),
    sum(when(col("dificuldade_classificada") == "Difícil", 1).otherwise(0)).alias("qtd_dificil"),
    
    # Proporções importantes
    round(avg(col("item_anulado")), 3).alias("proporcao_anulados"),
    round(avg(when(col("tipo_de_prova") == "Prova adaptada", 1).otherwise(0)), 3).alias("proporcao_adaptados")

)

df_gold.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("nttdataeducacao.gold.agg_itens_por_area")

display(df_gold)


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

### **Resumo por Cor da Prova**

In [0]:
df_gold_cor = df_itensProva_ouro.groupBy("cor_da_prova").agg(

    count("*").alias("total_itens"),
    
    sum(col("item_anulado")).alias("total_anulados"),
    
    sum(col("indicador_de_item_abandonado")).alias("total_invalidos"),
    
    sum(when(col("tipo_de_prova") == "Prova adaptada", 1).otherwise(0)).alias("total_adaptados"),
    
    round(avg("parametro_de_dificuldade"), 3).alias("media_dificuldade"),
    round(avg("parametro_de_discriminacao"), 3).alias("media_discriminacao"),
    round(avg("parametro_de_acerto_ao_acaso"), 3).alias("media_acerto_ao_acaso")
    
)

df_gold_cor.write.format("delta").mode("overwrite").saveAsTable("nttdataeducacao.gold.agg_por_cor")
display(df_gold_cor)



Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

### **Resumo por Tipo de Prova**

In [0]:
df_gold_tipo_prova = df_itensProva_ouro.groupBy("tipo_de_prova").agg(

    count("*").alias("total_itens"),
    
    sum(col("item_anulado")).alias("total_anulados"),

    sum(col("indicador_de_item_abandonado")).alias("total_invalidos"),

    round(avg("parametro_de_dificuldade"), 3).alias("media_dificuldade"),
    round(avg("parametro_de_discriminacao"), 3).alias("media_discriminacao"),
    round(avg("parametro_de_acerto_ao_acaso"), 3).alias("media_acerto_ao_acaso"),

    sum(when(col("dificuldade_classificada") == "Difícil", 1).otherwise(0)).alias("qtd_dificil"),
    sum(when(col("faixa_de_discriminacao") == "Alta", 1).otherwise(0)).alias("qtd_discriminacao_alta")

)

df_gold_tipo_prova.write.format("delta").mode("overwrite").saveAsTable("nttdataeducacao.gold.agg_por_tipo_de_prova")

display(df_gold_tipo_prova)



Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

### **Resumo por Faixa de Dificuldade**

In [0]:
df_gold_dificuldade = df_itensProva_ouro.groupBy("dificuldade_classificada").agg(

    count("*").alias("total_itens"),

    sum(col("item_anulado")).alias("total_anulados"),

    sum(col("indicador_de_item_abandonado")).alias("total_invalidos"),

    round(avg("parametro_de_dificuldade"), 3).alias("media_dificuldade"),
    round(avg("parametro_de_discriminacao"), 3).alias("media_discriminacao"),
    round(avg("parametro_de_acerto_ao_acaso"), 3).alias("media_acerto_ao_acaso"),

    sum(when(col("tipo_de_prova") == "Prova adaptada", 1).otherwise(0)).alias("total_adaptados")

)

df_gold_dificuldade.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("nttdataeducacao.gold.agg_por_dificuldade")

display(df_gold_dificuldade)



Databricks visualization. Run in Databricks to view.

In [0]:
df_itensProva_ouro.write.mode("overwrite").saveAsTable("nttdataeducacao.gold.itensProva_ouro")